# Sokoban assignment


The functions and classes defined in this module will be called by a marker script. 
You should complete the functions and classes according to their specified interfaces.

No partial marks will be awarded for functions that do not meet the specifications
of the interfaces. It would not be good to release a malfunctioning robot into a warehouse!

Please do NOT change the defined interfaces as we won't be able to test/mark your code.
In other words, you must fully adhere to the specifications of the 
functions, their arguments and returned values.
Changing the interface of a function will likely result in a fail 
for the test of your code. This is not negotiable! 

You have to make sure that your code works with the files provided 
(search.ipynb and sokoban.ipynb) as your code will be tested 
with the original copies of these files. 

Last modified by 2025-03-25  by will.browne@qut.edu.au 
- clarifiy some comments, renamed some functions
  (and hopefully didn't introduce any bug!)



In [9]:
# Set up and import useful functionality 

# https://saturncloud.io/blog/how-to-import-jupyter-notebooks-to-another-jupyter-notebook/
!pip install import_ipynb # should only need to run once
import import_ipynb # allows you to use other *.ipynb files
import search       # such as search tools written by course textbook and similar to week 3
import sokoban     # defines the problem domain
import itertools # please see https://docs.python.org/3/library/itertools.html
import time # please investigate external files

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -



Add the following: 

    Return the list of the team members of this assignment submission as a list
    of triplet of the form (student_number, first_name, last_name)

    Simply add in your student number, first name and last name for each team member in the format below
    and uncomment the line by removing the # [BTW please google the pioneers of computer science listed]
    



In [10]:
def my_team():
    return [ (11032553, 'Hunter', 'Wilde'), (12026395, 'Oliver', 'Kele') ]

Taboo cells:

    Identify the taboo cells of a warehouse. A "taboo cell" is by definition
    a cell inside a warehouse such that whenever a box get pushed on such 
    a cell then the puzzle becomes unsolvable. 
    
    Cells outside the warehouse are not taboo. It is a fail to tag one as taboo.
    
    When determining the taboo cells, you must ignore all the existing boxes, 
    only consider the walls and the target  cells.  
    Use only the following rules to determine the taboo cells;
     Rule 1: if a cell is a corner and not a target, then it is a taboo cell.
     Rule 2: all the cells between two corners along a wall are taboo if none of 
             these cells is a target.
    
    @param warehouse: 
    
        a Warehouse object with a worker inside the warehouse

    @return
    
       A string representing the warehouse with only the wall cells marked with 
       a '#' and the taboo cells marked with a 'X'.  
       The returned string should NOT have marks for the worker, the targets,
       and the boxes.  


In [11]:
# Hint: This is a great place to start the assignment
# Think about the 'out there' world in terms of a grid. 
# Inspect some of the given warehouses to determine their properties, e.g. are they all square or rectangular
# Are there certain cells in the grid that we should not consider in the problem domain, i.e. they are unreachable by the agent?
# Identify the subset of cells that should be investigated to whether they are Taboo or not.

# Move away from the computer for 15 minutes so that you can get a pen and paper: draw lots of sketches, diagrams and pseudocode (and then come back!)
# Write the pseudocode as comments

# Then build actual code under the comments testing as you go to check each line works as intended where possible
# Programming hint: Either write all code in function below or create and call new functions (e.g. CellReachability(*argument*))
# a small number of well-commented new functions is likely the easiest approach for clarity of code, which helps debugging

import re

def intialise_warehouse(file_path):

    # Function for loading a warehouse.txt file and creating a Warehouse object -> FOR TESTING PURPOSES ONLY
    
    wh = sokoban.Warehouse()
    wh.load_warehouse(file_path)

    return wh

def process_lines(warehouse_string):
    # Turn a string into lines
    # Replace all @ and $ with " " -> i.e. replace boxes and players with blanks
    # Replaces boxes on goal cells ('*') and players on goal cells ('!') with '.' -> i.e. ignore boxes and players, just mark goal cells

    lines = warehouse_string.splitlines()
    processed_lines = []

    for line in lines:
        # Scan across characters in the line and apply the regex replacements
        processed_line = re.sub(r'[@$]', ' ', re.sub(r'[!*]', '.', line))
        processed_lines.append(processed_line)
        
    return processed_lines
    

def find_corners(warehouse):

    wh_string = warehouse.__str__()
    lines = process_lines(wh_string)

    # Find all empty cells inside of the warehouse

    wall_pos = warehouse.walls # list of tuples [(x,y)...]

    # Record blank cells (inside the warehouse)

    blank_pos = [] # a list of tuples

    # Iterate over each line (a string) in lines (a list of strings)
    # For each line, iterate over each character in the string

    for y, line in enumerate(lines[1:(len(lines)-1)], 1): # don't look for corners in first or last line
        
        last_hash = len(line) - 1 - line[::-1].index('#')
        found_first_hash = False
        
        for x, char in enumerate(line[:last_hash]):
            if not found_first_hash:
                if char == " ": continue
                if char == "#": found_first_hash = True
            else:
                if char == " ": blank_pos.append((x,y)) # ignore target cells as they can not be taboo, and method for finding their positions
                
    
    # Find corners by comparing blank_pos to wall_pos
    corners = []

    # Directions to check for adjacent positions (left, right, up, down)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # (dx, dy) for left, right, up, down

    # Iterate over blank positions and check if they are corners
    for x, y in blank_pos:
        # Check if both vertical and horizontal directions have walls
        has_wall_x = (x - 1, y) in wall_pos or (x + 1, y) in wall_pos  # left or right
        has_wall_y = (x, y - 1) in wall_pos or (x, y + 1) in wall_pos  # up or down

        if has_wall_x and has_wall_y:
            corners.append((x, y))

    return corners

def find_corner_pairs(warehouse, corners):

    # Function to find series of cells between two corners with no target cells
    # Param: corners [(x,y)...]

    corner_pairs = []  # [(V/H, C1, C2)...] where C1 and C2 are (x,y) tuples
    targets_set = set(warehouse.targets)
    walls_set = set(warehouse.walls)
    target_or_wall_set = targets_set.union(walls_set)

    # for each corner
    for i in range(len(corners)):
        # unpack the corner's coordinates
        x1, y1 = corners[i]
        # look through the other corners
        for j in range(i + 1, len(corners)):
            # unpack the next corner's coordinates
            x2, y2 = corners[j]
            # record series of cells that could be taboo & whether to look horizontally or vertically
            if y1 == y2:
                # Horizontal scan
                x_start, x_end = sorted([x1, x2])
                has_target_or_wall = any((x, y1) in target_or_wall_set for x in range(x_start + 1, x_end))
                if not has_target_or_wall:
                    corner_pairs.append(("H", (x1, y1), (x2, y2)))

            elif x1 == x2:
                # Vertical scan
                y_start, y_end = sorted([y1, y2])
                has_target_or_wall = any((x1, y) in target_or_wall_set for y in range(y_start + 1, y_end))
                if not has_target_or_wall:
                    corner_pairs.append(("V", (x1, y1), (x2, y2)))

    return corner_pairs

def check_walls(warehouse, corners):

    # Function to check for taboo cells along a wall, based on corner_pairs passed in as a parameter

    corner_pairs = find_corner_pairs(warehouse, corners)

    along_wall = [] # list to store taboo cells (tuple of x and y) along a wall between two corners with no target cellls on the wall
    walls = warehouse.walls

    for corner_pair in corner_pairs:
        # Unpack corner pair and direction
        direction, C1, C2 = corner_pair
        x1, y1 = C1
        x2, y2 = C2

        if direction == "H":
            for x in range(x1 + 1, x2):
                # Check for walls above and below
                above = (x, y1-1) in walls
                below = (x, y1+1) in walls
                if above or below:
                    along_wall.append((x, y1))
                    
        elif direction == "V":
            y_start, y_end = sorted([y1, y2])
            for y in range(y_start + 1, y_end):
                # Check for walls to the left and right
                left = (x1 - 1, y) in walls
                right = (x1 + 1, y) in walls
                if left or right:
                    along_wall.append((x1, y))

    return along_wall

def find_taboo_cells(warehouse):

    # Function that returns a list of tuples (x,y) with the coordinates of all taboo cells

    taboo_cells = []
    
    corners = find_corners(warehouse)
    taboo_cells.extend(corners)
    
    between_corners = check_walls(warehouse, corners)
    taboo_cells.extend(between_corners)

    return taboo_cells

def taboo_cells(warehouse):

    taboo_cells = find_taboo_cells(warehouse)

    warehouse_string = warehouse.__str__()

    warehouse_lines = process_lines(warehouse_string) # Returns a list of lines with only " ", ".", and "#" (empty cell, target cell, and walls)

    # Replace target cells with blanks and insert "X" to mark taboo cells 

    new_lines = []

    for y, line in enumerate(warehouse_lines):
        new_line = ''
        for x, char in enumerate(line):
            if (x, y) in taboo_cells:
                new_line += 'X'
            elif char == '.':  # Replace target cells with blanks
                new_line += ' '
            else:
                new_line += char
        new_lines.append(new_line)

    return "\n".join(new_lines)

# TESTING TABOO CELLS FUNCTION

warehouse = intialise_warehouse("./warehouses/warehouse_6n.txt")
print(taboo_cells(warehouse))



 #### #### 
##XX###XX##
#X  #X#  X#
#X       X#
###     ###
 #X     X# 
###     X# 
#X      X# 
#X  #XXX## 
##XX####   
 ####      



    An instance of the class 'SokobanPuzzle' represents a Sokoban puzzle.
    An instance contains information about the walls, the targets, the boxes
    and the worker.

    Your implementation should be fully compatible with the search functions of 
    the provided module 'search.ipynb'

In [12]:
class SokobanPuzzle(search.Problem)

    #
    #         "INSERT YOUR CODE HERE"
    #
    #     Revisit the sliding puzzle and the pancake puzzle for inspiration!
    #
    #     Note that you will need to add several functions to 
    #     complete this class. For example, a 'result' method is needed
    #     to satisfy the interface of 'search.Problem'.
    #
    #     You are allowed (and encouraged) to use auxiliary functions and classes

    
    def __init__(self, warehouse):
        """
        Writes the lines of the warehouse file into the `rows` array;
        searches each row for the initial state (i.e. the player's position '@')
        & the goal state (i.e. the target square's position '.'),
        the indices of which are then stored in the `initial` and `goal` variables.
        :param warehouse: text file mapping the warehouse layout.
        """
        file = open(warehouse, 'r')
        rows = file.readlines()
        for row in rows:
            if '@' in row:
                self.initial = (row.index('@'), rows.index(row)) # tuple[int x,int y]
            if '.' in row:
                self.goal = (row.index('.'), rows.index(row)) # tuple[int x,int y]

    def actions(self, state):
        """
        Return the list of actions that can be executed in the given state.
        :param state: a given state.
        """
        raise NotImplementedError

SyntaxError: expected ':' (878311057.py, line 1)

    Determine if the sequence of actions listed in 'action_seq' is legal or not.
    
    Important notes:
      - a legal sequence of actions does not necessarily solve the puzzle.
      - an action is legal even if it pushes a box onto a taboo cell.
        
    @param warehouse: a valid Warehouse object

    @param action_seq: a sequence of legal actions.
           For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
           
    @return
        The string 'Impossible', if one of the action was not valid.
           For example, if the agent tries to push two boxes at the same time,
                        or push a box into a wall.
        Otherwise, if all actions were successful, return                 
               A string representing the state of the puzzle after applying
               the sequence of actions.  This must be the same string as the
               string returned by the method  Warehouse.__str__()

In [22]:
# Problem domains addressed by AI have *hard* and *soft* constraints
# Identify the hard constraints in this problem's definition

# Defining check functions, returns True if action legal, False if action illegal

# Direction deltas: (dx, dy)
deltas = {
    'L': (-1, 0),
    'R': (1, 0),
    'U': (0, -1),
    'D': (0, 1)
}

def get_player_destination(warehouse, action):

    x, y = warehouse.worker # worker position

    dx, dy = deltas[action]

    fx, fy = (x + dx, y + dy) # destination cell if action was performed

    return fx, fy

def legal_player_destination(fx, fy):

    if (fx, fy) in warehouse.walls: return False
    else: return True

def is_box(fx, fy, warehouse):

    if (fx, fy) in warehouse.boxes: return True
    else: return False

def legal_box_destination(warehouse, action):

    # Check if there's a box in the cell the player is moving into

    x, y = warehouse.worker # worker position

    dx, dy = deltas[action]

    fx, fy = (x + 2*dx, y + 2*dy) # destination cell if box was pushed

    # Check the box's destination and determine whether the action is legal

    boxes_set = set(warehouse.boxes)
    walls_set = set(warehouse.walls)
    box_or_wall_set = boxes_set.union(walls_set)

    if (fx, fy) in box_or_wall_set: return False
    else: return True

def legal_action(warehouse, action):
    
    fx, fy = get_player_destination(warehouse, action)

    if legal_player_destination(fx, fy):
        if is_box(fx, fy, warehouse): 
            return legal_box_destination(warehouse, action)
        else: return True
    else:
        return False

def update_warehouse(warehouse, action):
    # This function updates a warehouse given a LEGAL action (function assumes an action is legal)

    new_warehouse = sokoban.Warehouse()

    fx, fy = get_player_destination(warehouse, action)

    if is_box(fx, fy, warehouse):
        
        new_boxes = warehouse.boxes
        moved_box = fx, fy

        dx, dy = deltas[action]
        hx, hy = fx + dx, fy + dy

        new_boxes.remove((fx, fy))
        new_boxes.append((hx, hy))

        new_warehouse = warehouse.copy(worker = (fx, fy), boxes = new_boxes)
    else:
        new_warehouse = warehouse.copy(worker = (fx, fy))
        
    return new_warehouse


def check_elem_action_seq(warehouse, action_seq):

    current_warehouse = warehouse

    for action in action_seq:
        if legal_action(current_warehouse, action):
            current_warehouse = update_warehouse(current_warehouse, action)
        else:
            return "Impossible"

    return current_warehouse.__str__()


# TESTING

# Testing Player Destination Check

warehouse = intialise_warehouse("./warehouses/warehouse_01.txt")
print(warehouse.__str__())
print(legal_player_destination(warehouse, "R"))

# Testing Box Destination Check

print(legal_box_destination(warehouse, "L"))

# Testing Check Action

print(legal_action(warehouse, "U"))

# Testing Update Warehouse

wh = update_warehouse(warehouse, "D")
print(update_warehouse(wh, "R"))

# Testing Check_Elem_Action_Seq

print(check_elem_action_seq(warehouse, ["D","R","L"]))


####  
#  #  
#  ###
#*@  #
#  $.#
#  ###
####  
True
False
True
####  
#  #  
#  ###
#*   #
#  @*#
#  ###
####  
####  
#  #  
#  ###
#*   #
# @ *#
#  ###
####  


    This function analyses the given warehouse.
    It returns the two items. The first item is an action sequence solution. 
    The second item is the total cost of this action sequence.
    
    @param 
     warehouse: a valid Warehouse object

    @return
    
        If puzzle cannot be solved 
            return 'Impossible', None
        
        If a solution was found, 
            return S, C 
            where S is a list of actions that solves
            the given puzzle coded with 'Left', 'Right', 'Up', 'Down'
            For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
            If the puzzle is already in a goal state, simply return []
            C is the total cost of the action sequence C

In [ ]:
# Now consider the weights of the boxes being pushed
# Determine how this changes the cost function in the graph search

def solve_weighted_sokoban(warehouse):

    
    raise NotImplementedError()